In [1]:
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

### Train

In [2]:
train = pd.read_csv('../../ml-data/cyberattack_detection/3_selected_features/A1_train_data_VT.csv', sep=',')

In [3]:
train.head()

,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,Protocol,SrcIPAddress,SrcPort,DestIPAddress,DestPort,State,DestTypeService
0,-0.008024,-0.008615,-0.003897,-0.296301,-0.308511,-0.141071,11.0,5663.0,123467.0,475015.0,57944.0,120.0,0.0
1,-0.010103,-0.008919,-0.004151,-0.296351,-0.308520,-0.141076,11.0,5663.0,123658.0,46220.0,34823.0,327.0,4.0
2,-0.010363,-0.008918,-0.004149,-0.296351,-0.308520,-0.141076,11.0,5663.0,123546.0,76319.0,34823.0,327.0,4.0
3,-0.010363,-0.008917,-0.004151,-0.296351,-0.308520,-0.141076,11.0,5663.0,123606.0,452301.0,34823.0,327.0,4.0
4,-0.010623,-0.008919,-0.004149,-0.296351,-0.308520,-0.141076,11.0,5663.0,123487.0,79211.0,34823.0,327.0,4.0


In [4]:
# set isolation = 0.015
isolation_forest = IsolationForest(contamination=0.015)

In [5]:
isolation_forest.fit(train)

IsolationForest(contamination=0.015)

### Test

In [6]:
test = pd.read_csv('../../ml-data/cyberattack_detection/3_selected_features/A1_test_data_VT.csv', sep=',')

In [7]:
test_predict = isolation_forest.predict(test)
decision_function = isolation_forest.decision_function(test)

In [8]:
test['anomaly'] = test_predict
test['score'] = decision_function

In [9]:
test['anomaly'].value_counts()

 1    1035922
-1      17923
Name: anomaly, dtype: int64

In [10]:
#sns.pairplot(test, hue = 'anomaly')

In [11]:
anomalies = test.loc[test['anomaly'] == -1]

In [12]:
anomalies.head()

,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,Protocol,SrcIPAddress,SrcPort,DestIPAddress,DestPort,State,DestTypeService,anomaly,score
249,0.386255,0.306761,0.016051,-0.260652,0.008747,-0.128602,11.0,3047.0,46659.0,497552.0,83669.0,120.0,0.0,-1,-0.081829
410,-0.010363,-0.008897,-0.004102,0.503174,0.324613,0.742554,12.0,9599.0,94587.0,579324.0,93884.0,165.0,4.0,-1,-0.001361
1969,0.210038,0.205792,0.007113,-0.240186,0.302055,-0.121371,11.0,1145.0,46473.0,479317.0,95103.0,308.0,0.0,-1,-0.092632
2076,0.051495,0.044501,0.000387,-0.295605,-0.301774,-0.140721,11.0,1145.0,47165.0,405258.0,95103.0,308.0,0.0,-1,-0.021504
2089,0.033301,0.026225,0.000295,-0.295927,-0.305121,-0.140810,11.0,1145.0,47373.0,630570.0,95103.0,308.0,0.0,-1,-0.012252


In [13]:
original_test = pd.read_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_1/clean_test_data', sep=',')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
anomalies_orig_feat = original_test[original_test.index.isin(anomalies.index)]

In [15]:
anomalies_orig_feat = anomalies_orig_feat.merge(anomalies[['score']], left_index = True, right_index=True)

In [16]:
anomalies_orig_feat.head(10)

,Timestamp,Duration,Protocol,SrcIPAddress,SrcPort,Direction,DestIPAddress,DestPort,State,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,score
249,2012/12/01 13:34:30.899435,1.850078,tcp,153.217.92.157,1040,->,207.177.143.150,5678,FSPA_FSPA,0.0,0.0,1528,1201775,33613,825.911124,6.495807e+05,1.816842e+04,-0.081829
410,2012/12/01 13:38:06.725212,0.000108,udp,224.214.187.244,33676,->,221.34.91.52,6881,INT,0.0,NaN,2,140,139,18518.518519,1.296296e+06,1.287037e+06,-0.001361
1969,2012/12/01 14:14:22.366222,0.653870,tcp,132.200.95.181,1030,->,204.136.12.18,80,SRPA_FSPA,0.0,0.0,850,817410,18766,1299.952590,1.250111e+06,2.869989e+04,-0.092632
2076,2012/12/01 14:15:39.871327,14.729237,tcp,132.200.95.181,1066,->,191.117.240.239,80,SRPA_FSPA,0.0,0.0,240,203414,7596,16.294123,1.381022e+04,5.157090e+02,-0.021504
2089,2012/12/01 14:15:40.060378,19.236521,tcp,132.200.95.181,1077,->,230.40.252.239,80,SRPA_FSPA,0.0,0.0,170,133841,7442,8.837357,6.957651e+03,3.868683e+02,-0.012252
2167,2012/12/01 14:15:57.442042,81.530454,tcp,132.200.95.181,1122,->,135.85.246.76,80,SRPA_SPA,0.0,0.0,3606,2979963,71307,44.228872,3.655031e+04,8.746057e+02,-0.099759
2173,2012/12/01 14:16:01.911863,38.167989,tcp,132.200.95.181,1125,->,134.238.251.2,80,FSPA_FSPA,0.0,0.0,8322,7401735,199720,218.036114,1.939252e+05,5.232657e+03,-0.105256
2291,2012/12/01 14:17:08.878287,3.743253,udp,132.200.95.181,7600,<->,125.16.161.15,7600,CON,0.0,0.0,189,175411,4268,50.490843,4.686058e+04,1.140185e+03,-0.001430
2319,2012/12/01 14:17:09.445501,3.601064,udp,132.200.95.181,7600,<->,128.144.109.215,7600,CON,0.0,0.0,284,243539,5802,78.865580,6.762973e+04,1.611190e+03,-0.029836
2325,2012/12/01 14:17:09.637860,4.113676,udp,132.200.95.181,7600,<->,172.16.128.10,7600,CON,0.0,0.0,332,267338,6507,80.706405,6.498762e+04,1.581797e+03,-0.032267


In [17]:
import numpy as np
anomalies_orig_feat.loc[anomalies_orig_feat['PacketsSec'] != np.inf].describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,Duration,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,score
count,17923,17918,2620,17923,17923,17923,17923,17923,17923,17923
mean,792.464,0,0,1599.07,1.34895e+06,1.13221e+06,4638.42,341173,157129,-0.0291216
std,530.415,0,0,31709.4,1.62902e+07,1.541e+07,37765.6,2.86119e+06,1.39992e+06,0.0318624
min,4e-06,0,0,1,57,0,0.00326689,1.05023,0,-0.190311
25%,347.201,0,0,18,18014,17380.5,0.0270111,27.2618,26.9978,-0.0331502
50%,934.029,0,0,29,29783,28251,0.0377628,38.9191,38.6393,-0.0207998
75%,1031.45,0,0,39,40392,38408.5,0.196118,192.648,173.043,-0.00925472
max,3858.04,0,0,3.68056e+06,9.45069e+08,9.83047e+08,1e+06,8.475e+07,4.492e+07,-1.39884e-06


In [18]:
anomalies_orig_feat['SrcIPAddress'].nunique()

133

In [19]:
anomalies_orig_feat.to_csv('../4 Results/A1_test_iForest0.015_VT_orig_feat.csv', sep=',')

End